<a href="https://colab.research.google.com/github/biranchi2018/My-ComputerVision-Examples/blob/master/1.APNR_yolo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## APNR using Yolo5


https://towardsdatascience.com/indian-car-license-plate-detection-using-yolo-v5-ae2574578175

https://github.com/sid0312/anpr_yolov5

In [ ]:
!git clone https://github.com/sid0312/anpr_yolov5

Cloning into 'anpr_yolov5'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 626 (delta 23), reused 24 (delta 4), pack-reused 569
Receiving objects: 100% (626/626), 64.30 MiB | 31.18 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
%cd anpr_yolov5/

/content/anpr_yolov5


In [ ]:
!python detect.py --source sample_cars/  --weights weights/last.pt --conf 0.4 --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', fourcc='mp4v', img_size=640, iou_thres=0.5, output='inference/output', save_txt=True, source='sample_cars/', view_img=False, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

image 1/5 sample_cars/car_0.jpg: 448x640 1 license_plates, Done. (0.028s)
image 2/5 sample_cars/car_1.jpg: 384x640 Done. (0.025s)
image 3/5 sample_cars/car_2.jpg: 512x640 1 license_plates, Done. (0.029s)
image 4/5 sample_cars/car_3.jpg: 448x640 1 license_plates, Done. (0.027s)
image 5/5 sample_cars/car_4.jpg: 512x640 2 license_plates, Done. (0.029s)
Results saved to /content/anpr_yolov5/inference/output
Done. (0.317s)


In [ ]:
import os
print(os.listdir('/content/anpr_yolov5/inference/output'))

['car_0.jpg', 'car_1.jpg', 'car_3.jpg', 'car_2.jpg', 'car_4.jpg']


In [ ]:
from google.colab import files

In [ ]:
for img in os.listdir('/content/anpr_yolov5/inference/output'):
  path = os.path.join('/content/anpr_yolov5/inference/output',img)
  files.download(path)


In [ ]:
!cat detect.py

import argparse

from utils.datasets import *
from utils.utils import *


def detect(save_img=False):
    out, source, weights, view_img, save_txt, imgsz = \
        opt.output, opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size
    webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

    # Initialize
    device = torch_utils.select_device(opt.device)
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    google_utils.attempt_download(weights)
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    # torch.save(torch.load(weights, map_location=device), weights)  # update model if SourceChangeWarning
    # model.fuse()
    model.to(device).eval()
    if half:
        model.half()  # to FP16

    # Second-stage classifier